In [19]:
#Importing necessary libraries

import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [16]:
#Defining all the required functions

#Function to extract Product Name 
def get_pdt_name(soup):

    try:
        pdt_name_string = soup.find("span", attrs = {"id": 'productTitle'}).text.strip()

    except AttributeError:
        pdt_name_string = ""

    return pdt_name_string

# Function to extract Product Price
def get_pdt_price(soup):

    try:
        pdt_price = soup.find("span", attrs = {'class': 'a-price aok-align-center reinventPricePriceToPayMargin priceToPay'}).find("span", attrs = {'class': 'a-price-whole'}).text.strip()

    except AttributeError:
        pdt_price = ""

    return pdt_price

# Function to extract Product Rating
def get_pdt_rating(soup):

    try:
        pdt_rating = soup.find("span", attrs = {'class': 'a-icon-alt'}).text.strip()
    
    except AttributeError:
        pdt_rating = ""
            
    return pdt_rating

# Function to extract Seller Name
def get_pdt_seller_name(soup):
    try:
        pdt_seller_name = soup.find("div", attrs = {"id": 'merchant-info'}).find("a", attrs = {"class": 'a-link-normal'}).find("span").text

    except AttributeError:
        pdt_seller_name = ""

    return pdt_seller_name

In [18]:
if __name__ == '__main__':
    
    # add our user agent 
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})

    # The URL of webpage to be extracted
    url = "https://www.amazon.in/s?rh=n%3A6612025031&fs=true&ref=lp_6612025031_sar"

    # HTTP Request
    req = requests.get(url, headers = HEADERS)

    # Soup Object containing all the data
    soup1 = BeautifulSoup(req.content, "html.parser")

    # Fetch links as List of Tag Objects
    pdt_links = soup.find_all("a", attrs = {'class': 'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})

    # Storing the links of each product
    pdt_links_list = []

    # Loop for extracting links from Tag Objects defined earlier
    for pdt_link in pdt_links:
            pdt_links_list.append(pdt_link.get('href'))

    #Defining dictionary to store the required data
    dict1 = {"PDT_NAME": [], "PDT_PRICE": [], "PDT_RATING":[], "PDT_SELLER_NAME": []}
    
    # Loop for extracting product details from each link 
    for pdt_link in pdt_links_list:
        new_req = requests.get("https://www.amazon.in" + pdt_link, headers = HEADERS)
        new_soup1 = BeautifulSoup(new_req.content, "html.parser")

        # Function calls to display all necessary product information
        dict1['PDT_NAME'].append(get_pdt_name(new_soup1))
        dict1['PDT_PRICE'].append(get_pdt_price(new_soup1))
        dict1['PDT_RATING'].append(get_pdt_rating(new_soup1))
        dict1['PDT_SELLER_NAME'].append(get_pdt_seller_name(new_soup1))
    
    #Converting dictionary to csv file
    amazon_data_df = pd.DataFrame.from_dict(dict1)
    amazon_data_df['PDT_NAME'].replace('', np.nan, inplace = True)
    amazon_data_df = amazon_data_df.dropna(subset = ['PDT_NAME'])
    amazon_data_df.to_csv("amazon_data_df_pdt.csv", header = True, index = False)